# DSCI 525 - Web and Cloud Computing

# Milestone 4 - Group 8

## Rachel Wong (Student 81), Rui Wang (Student 76), Daniel Ortiz (Student 50), Santiago Rugeles Schoonewolff (Student 57)

Repo link: https://github.com/UBC-MDS/525-group8

***Milestone 4:*** In this milestone, you will deploy the machine learning model you trained in milestone 3.

Milestone 4 checklist :

- [ ] Use an EC2 instance.
- [ ] Develop your API here in this notebook.
- [ ] Copy it to ```app.py``` file in EC2 instance.
- [ ] Run your API for other consumers and test among your colleagues.
- [ ] Summarize your journey.

In [3]:
## Import all the packages that you need
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

## 1. Develop your API

rubric={mechanics:45}

You probably got how to set up primary URL endpoints from the ```sampleproject.ipynb notebook``` and have them process and return some data. Here we are going to create a new endpoint that accepts a POST request of the features required to run the machine learning model that you trained and saved in last milestone (i.e., a user will post the predictions of the 25 climate model rainfall predictions, i.e., features,  needed to predict with your machine learning model). Your code should then process this data, use your model to make a prediction, and return that prediction to the user. To get you started with all this, I've given you a template which you should fill out to set up this functionality:

***NOTE:*** You won't be able to test the flask module (or the API you make here) unless you go through steps in ```2. Deploy your API```. However, here you can make sure that you develop all your functions and inputs properly.

```python
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

# 1. Load your model here
model = joblib.load(...)

# 2. Define a prediction function
def return_prediction(...):

    # format input_data here so that you can pass it to model.predict()

    return model.predict(...)

# 3. Set up home page using basic html
@app.route("/")
def index():
    # feel free to customize this if you like
    return """
    <h1>Welcome to our rain prediction service</h1>
    To use this service, make a JSON post request to the /predict url with 5 climate model outputs.
    """

# 4. define a new route which will accept POST requests and return model predictions
@app.route('/predict', methods=['POST'])
def rainfall_prediction():
    content = request.json  # this extracts the JSON content we sent
    prediction = return_prediction(...)
    results = {...}  # return whatever data you wish, it can be just the prediction
                     # or it can be the prediction plus the input data, it's up to you
    return jsonify(results)
```

## 2. Deploy your API

rubric={mechanics:40}

Once your API (app.py) is working we're ready to deploy it! For this, do the following:

1. SSH into your EC2 instance from milestone2. There are no issues if you want to spin another EC2 instance; if you plan to do so, make sure you terminate any other running instances.
2. Make a file `app.py` file in your instance and copy what you developed above in there. 

    2.1 You can use the linux editor using ```vi```. More details on vi Editor [here](https://www.guru99.com/the-vi-editor.html). I do recommend doing it this way and knowing some basics like ```:wq,:q!,dd``` will help.
    
    2.2 Or else you can make a file in your laptop called app.py and copy it over to your EC2 instance using ```scp```. Eg: ```scp -r -i "ggeorgeAD.pem" ~/Desktop/worker.py  ubuntu@ec2-xxx.ca-central-1.compute.amazonaws.com:~/```

3. Download your model from s3 to your EC2 instance.
4. Presumably you already have `pip` or `conda` installed on your instance from your previous milestone. You should use one of those package managers to install the dependencies of your API, like `flask`, `joblib`, `sklearn`, etc.

    4.1. You have installed it in your TLJH using [Installing pip packages](https://tljh.jupyter.org/en/latest/howto/env/user-environment.html#installing-pip-packages). if you want to make it available to users outside of jupyterHub (which you want to in this case as we are logging into EC2 instance as user ```ubuntu``` by giving ```ssh -i privatekey ubuntu@<host_name>```) you can follow these [instructions](https://tljh.jupyter.org/en/latest/howto/env/user-environment.html#accessing-user-environment-outside-jupyterhub).
    
    4.2. Alternatively you can install the required packages inside your terminal.
        - Install conda:
            wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
            bash Miniconda3-latest-Linux-x86_64.sh
        - Install packages (there might be others): 
            conda install flask scikit-learn joblib

5. Now you're ready to start your service, go ahead and run `flask run --host=0.0.0.0 --port=8080`. This will make your service available at your EC2 instance's IP address on port 8080. Please make sure that you run this from where ```app.py``` and ```model.joblib``` resides.
6. You can now access your service by typing your EC2 instances public IPv4 address appened with `:8080` into a browswer, so something like `http://<your_EC2_ip>:8080`.
7. You should use `curl` to send a post request to your service to make sure it's working as expected.
>EG: curl -X POST http://your_EC2_ip:8080/predict -d '{"data":[1,2,3,4,53,11,22,37,41,53,11,24,31,44,53,11,22,35,42,53,12,23,31,42,53]}' -H "Content-Type: application/json"

8. Now, what happens if you exit your connection with the EC2 instance? Can you still reach your service?
9. There are several options we could use to help us persist our server even after we exit our shell session. We'll be using `screen`. `screen` will allow us to create a separate session within which we can run `flask` and which won't shut down when we exit the main shell session. Read [this](https://linuxize.com/post/how-to-use-linux-screen/) to learn more on ```screen```.
10. Now, create a new `screen` session (think of this as a new, separate shell), using: `screen -S myapi`. If you want to list already created sessions do ```screen -list```. If you want to get into an existing ```screen -x myapi```.
11. Within that session, start up your flask app. You can then exit the session by pressing `Ctrl + A then press D`. Here you are detaching the session, once you log back into EC2 instance you can attach it using ```screen -x myapi```.
12. Feel free to exit your connection with the EC2 instance now and try accessing your service again with `curl`. You should find that the service has now persisted!
13. ***CONGRATULATIONS!!!*** You have successfully got to the end of our milestones. Move to Task 3 and submit it.

## 3. Summarize your journey from Milestone 1 to Milestone 4
rubric={mechanics:10}
>There is no format or structure on how you write this. (also, no minimum number of words).  It's your choice on how well you describe it.

It has certainly been quite a new and interesting journey. None of us had used these web services nor deployed a model. From our first team meeting, we kicked it off great and our main learning objectives for this course were learning about cloud and web services and getting familiar with AWS interface. 

We put on our Data Engineer hat for Milestone 1. It was a great way to start realizing the limits of our laptops and how useful cloud services are, especially working with big data. Since we were running the code in the local machine, it took a long time to run. We combatted these problems by having to restart from scratch if there's anything we want to modify from the start which was frustrating. Everytime we reran the notebook, we had to delete the downloaded files and redownload it again, which is quite challenging for large-scale data processing. As we only had one single machine, our EDA was very simple, we could hardly visualize our data or calculate correlation matrices. We were unable to do deep EDA like plots (histograms, correlation matrices, etc.) to show relationships between features because the data was so large and taking a sample of the data would not be ideal. To combat this we focused on simpler EDA methods such as analyzing mean/median/mode and count values.

In Milestone 2 we got hands on AWS with our Infrastructure hat on. It was somehow confusing but understandable given that it was our first time using it. We all created an EC2 instance to practice. After that we focused on student81 EC2 t2.large instance with JupyterHub. We also set up the S3 bucket to move our data from M1 to S3. We learned different storage options and knowing that we just needed pretty much to upload once, S3 bucket was ideal. After that, we prepared the data for our ML predictive model. This part was not that new to us but making sure we could all see the file in JupyterHub was tricky.

During Milestone 3, setting up the EMR cluster was pretty straight forward. However, setting up the browser, Jupyter environment and connecting to master node, was also somewhat tricky. Not really difficult but we encountered a few issues that we were fortunate to solve in the spot. After that, with our Data Scientist hat, developing the ML model with scikit-learn was more familiar to us as opposed to obtaining the best hyperparameter setting using spark’s MLlib.

Finally Milestone 4, and we wore our DevOps hat. We had our model ready and it was time to deploy it. We encountered many issues when trying to connect to the public IP. We did not know exactly why was it but in the end it was just a mistake on our side using the wrong EC2 instance with the wrong Security group. Once we passed that, everything else went smoothly and we were able to successfully deploy our model.

Overall, we believe it was a great experience to use all those different hats and have a better sense of a full process. Indeed we felt better with the Data Scientist hat on but it was very interesting to use the other three. It is always nice to be able to complete a project from beginning to end with the instructor and TA’s guidance. We will keep the Data Scientist hat on but will not hesitate to use any other hat when needed.


## 4. Submission instructions
rubric={mechanics:5}

In the textbox provided on Canvas please put a link where TAs can find the following-
- [ ] This notebook with solution to ```1 & 3```
- [ ] Screenshot from 
    - [ ] Output after trying curl. Here is a [sample](https://github.ubc.ca/MDS-2020-21/DSCI_525_web-cloud-comp_students/blob/master/Milestones/milestone4/images/curl_deploy_sample.png). This is just an example; your input/output doesn't have to look like this, you can design the way you like. But at a minimum, it should show your prediction value.